In [11]:
from magno import deeptrack as dt

In [12]:
encoder = dt.models.CTMAGIK(output_type="cls_rep")
teacher = dt.models.CTMAGIK(output_type="cls_rep")

encoder.summary()

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
node_ide1 (Dense)               (None, None, 32)     128         input_18[0][0]                   
__________________________________________________________________________________________________
lambda (Lambda)                 multiple             0           node_ide1[0][0]                  
                                                                

In [13]:
model = dt.models.MAGNO(encoder, teacher, representation_size=128)